## Integrate the particles where they feel all NBody forces

In [1]:
import tstrippy
from gcs import path_handler as ph
import gcs
import os 
import numpy as np 
import astropy.units as u

In [2]:
GCname              =   "NGC104"
montecarlokey       =   "monte-carlo-000"
internal_dynamics   =   "isotropic-plummer"
potential           =   "pouliasis2017pii-GCNBody"
NP                  =   int(1e5)
T                   =   5e9*u.yr
dt                  =   1e3*u.yr
unitT               =   u.s * u.kpc / u.km

load the particle positions

In [3]:
fname = ph.ParticleInitialConditions(GCname)
x,y,z,vx,vy,vz=gcs.extractors.ParticleInitialConditions.load_particles(fname,internal_dynamics,montecarlokey,NP)

load the host orbit 

In [4]:
# now get the positions at the begining of the orbit 
orbit_file_name = ph.GC_orbits(potential,GCname)
tH,xH,yH,zH,vxH,vyH,vzH=gcs.extractors.GCOrbits.extract_whole_orbit(orbit_file_name,montecarlokey)
t0,x0,y0,z0,vx0,vy0,vz0=gcs.extractors.GCOrbits.initial_simulation_coordinates(orbit_file_name,montecarlokey)

load the host's Mass and size

In [7]:
rh_m[0]

<Quantity 0.00644 kpc>

In [17]:
Mass,rh_m,_a,_,_,_,_,_=gcs.extractors.MonteCarloObservables.extract_all_GC_observables([GCname],montecarlokey)
rplummer=gcs.misc.half_mass_to_plummer(rh_m[0]).value
mass_host = Mass[0].value

load all the perturbers 

In [18]:
GCnames=list(tstrippy.Parsers.baumgardtMWGCs().data['Cluster'][:])
GCnames.remove(GCname)

In [19]:
ts,xs,ys,zs,vxs,vys,vzs=gcs.extractors.GCOrbits.extract_orbits_from_all_GCS(GCnames,potential,montecarlokey)

In [20]:
Masses,rh_mes,RAes,DECes,Rsunes,RVes,mualphaes,mu_deltaes=gcs.extractors.MonteCarloObservables.extract_all_GC_observables(GCnames,montecarlokey)
r_plums = [gcs.misc.half_mass_to_plummer(rh_m).value for rh_m in rh_mes]
Masses = [Mass.value for Mass in Masses]

In [21]:
tnew=np.arange(0,T.value+dt.value,dt.value)*u.yr
tnew=tnew - T
tnew=tnew.to(unitT).value

In [22]:
xH,yH,zH,vxH,vyH,vzH=gcs.misc.interpolate_finer_grid(tnew,tH,xH,yH,zH,vxH,vyH,vzH)

(5000001,)